# Lab 6: Visualizing the training process of neural networks. Hyperparameter tuning.

In this lab, you will learn how to use [wandb](https://wandb.ai/) to visualize the training process of neural networks. We are going to build and train a feed-forward neural network for recognizing handwritten digits of the MNIST dataset. The training process will be visualized in the wandb dashboard, which will allow us to monitor the loss and accuracy of the model in real-time.

---



Feel free to create an account at [wandb.ai](https://wandb.ai/) before starting this lab.

### A simple example of how to use wandb in a typical training loop is shown below:

```python
import wandb

wandb.login() # Log in to your wandb account

# Start a new run

some_config = {
    'learning_rate': 0.01,
    'layer_1_size': 128,
    'layer_2_size': 64,
    'batch_size': 32
} # This is just an example of a configuration dictionary, you can put anything you want here

wandb.init(project='mnist-classifier', config=some_config) # start a new run and log parameters

# Here you would prepare your data, and initialize the model, optimizer, etc.

# Training loop
for epoch in range(100):
    ...
    wandb.log({'loss': loss, 'accuracy': accuracy})
    # This will send the loss and accuracy to wandb and you can visualize it in the dashboard

# End of the run
wandb.finish()
```

The most important part is the `wandb.log()` function, which sends the data to the wandb dashboard. You can log any metric you want, not just loss and accuracy. The value passed to the function must be a dictionary.


## Exercise 1: Prepare data for training a mnist classifier (2 points)

Before you start training a neural network, you need to prepare the data. In this exercise, you will prepare the MNIST dataset of handwritten digits for training a classifier. You should:

1. Load the MNIST dataset using from `data/mnist_train.csv` and `data/mnist_test.csv` files.
2. Normalize the data to the range [0, 1].
3. Encode the labels using one-hot encoding.
4. Create a PyTorch `Dataset` object for the training and test sets.
5. Create a PyTorch `DataLoader` object for the training and test sets.

In [2]:
import pandas as pd

'''
The mnist_train.csv file contains the 60,000 training examples and labels. 
The mnist_test.csv contains 10,000 test examples and labels. Each row consists of 785 values: the first value is the label 
(a number from 0 to 9) and the remaining 784 values are the pixel values (a number from 0 to 255).
'''
# Load the MNIST dataset
train_data = pd.read_csv('data/mnist_train.csv') 
test_data = pd.read_csv('data/mnist_test.csv')


In [3]:
train_data.iloc[:,0]

0        0
1        4
2        1
3        9
4        2
        ..
59994    8
59995    3
59996    5
59997    6
59998    8
Name: 5, Length: 59999, dtype: int64

In [4]:
# Normalize the data to the range [0, 1]
train_data.iloc[:, 1:] = train_data.iloc[:, 1:] / 255.0
test_data.iloc[:, 1:] = test_data.iloc[:, 1:] / 255.0

In [5]:
# Encode the labels using one-hot encoding
train_labels = pd.get_dummies(train_data.iloc[:, 0])
test_labels = pd.get_dummies(test_data.iloc[:, 0])

train_labels

,0,1,2,3,4,5,6,7,8,9
0,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1
4,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
59994,0,0,0,0,0,0,0,0,1,0
59995,0,0,0,1,0,0,0,0,0,0
59996,0,0,0,0,0,1,0,0,0,0
59997,0,0,0,0,0,0,1,0,0,0


In [6]:
import torch

# Convert data and labels to tensors
train_data_tensor = torch.tensor(train_data.iloc[:, 1:].values, dtype=torch.float32)
train_labels_tensor = torch.tensor(train_labels.values, dtype=torch.float32)
test_data_tensor = torch.tensor(test_data.iloc[:, 1:].values, dtype=torch.float32)
test_labels_tensor = torch.tensor(test_labels.values, dtype=torch.float32)

# Create the train and test datasets
train_dataset = torch.utils.data.TensorDataset(train_data_tensor, train_labels_tensor)
test_dataset = torch.utils.data.TensorDataset(test_data_tensor, test_labels_tensor)


In [7]:
from torch.utils.data import DataLoader

# Create the train and test dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f'Train batches: {len(train_dataloader)}')
print(f'Test batches: {len(test_dataloader)}')


Train batches: 1875
Test batches: 313


In [8]:
# we can iterate over the dataloader to see how it works

X, y = next(iter(train_dataloader))

print("X is a tensor of shape:", X.shape)
print("y is a tensor of shape:", y.shape)

X is a tensor of shape: torch.Size([32, 784])
y is a tensor of shape: torch.Size([32, 10])


## Exercise 2: Prepare the architecture of the neural network (2 points)

In this exercise, you will prepare the architecture of the neural network. You should:

1. Create a neural network class that inherits from `torch.nn.Module`.
2. The neural network should have at least one hidden layer.
3. Use ReLU activation functions after each but the output layer.
4. Use a softmax activation function in the output layer to get the probabilities of each class.

**Feel free to experiment with the architecture of your network** - try adding more hidden layers, changing the number of neurons in each layer, etc. You can also add a dropout layer or some other regularization technique and see if it improves the performance of your model.

In [9]:
import torch.nn as nn
import torch.nn.functional as F

class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden1_size, hidden2_size, hidden3_size, output_size, dropout_prob=0.0):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden1_size)
        self.fc2 = nn.Linear(hidden1_size, hidden2_size)
        self.fc3 = nn.Linear(hidden2_size, hidden3_size)
        self.fc4 = nn.Linear(hidden3_size, output_size)
        self.dropout = nn.Dropout(dropout_prob)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = F.softmax(self.fc4(x), dim=1)
        return x

# Initialize the model with input size 784 (28x28 images), hidden sizes 128, 64, 32, output size 10 (number of classes), and dropout probability 0.0
model = NeuralNetwork(input_size=784, hidden1_size=128, hidden2_size=64, hidden3_size=32, output_size=10, dropout_prob=0.0)
print(model)

# Forward pass with a sample input tensor X
output = model(X)
print("Output shape:", output.shape)

NeuralNetwork(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=10, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
)
Output shape: torch.Size([32, 10])


In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## *Training PyTorch models on GPU

**GPUs are optimized for performing matrix operations in parallel.** Although we call them "graphics processing units", they are actually very powerful processors that can be used for any kind of parallel computation, including training deep neural networks. In fact, data science is one of the most common applications of GPUs today, as can be seen by the revenue of companies like NVIDIA over the past few years. NVIDIA is a monopolist in the GPU market - in 2023, the company owned 92% of the data center GPU market share. As for 31 July, the 2024 revenue of NVIDIA was 60.92 billion USD, while the total revenue of 2020 was $10.92 billion. If someone benefits from the current deep learning hype, it is certainly NVIDIA.

If you happen to have an NVIDIA GPU in your computer, you can use it to train your deep learning models, as PyTorch has excellent support for CUDA, which is NVIDIA's parallel computing API. To train a model on GPU, you need to explicitly tell PyTorch to move the model and the data to the GPU. 

Here is an example training loop that uses the GPU:

```python
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')   # Check if a GPU is available

# Initialize the model and move it to the GPU
model = SomeNeuralNetwork().to(device)   # Move the model to the GPU

for epoch in range(100):
    for batch in data_loader:
        X, y = batch
        X, y = X.to(device), y.to(device)   # Move the tensors to GPU
        
        y_pred = model(X)   # Perform a forward pass (on the GPU)
        loss = criterion(y_pred, y)   # Compute the loss (still on the GPU)
        
        ...  # The rest of the training loop
        
        y_pred = y_pred.detach().cpu()   # Move the predictions back to the CPU to do anything else with them
```

Note that **the model and all the tensors it uses for computation should be moved to the GPU**. You can do this by calling the `.to(device)` method on the model and the data tensors. If you want to move the data back to the CPU (to process it further, calculate metrics, visualize), you call the `.cpu()` method on the tensor.

**Doing calculations on the GPU, you should be wary of few things:**

* **The GPU has a limited amount of memory**, so you should be careful not to run out of memory. A typical graphics card has a few gigabytes of memory, so you should be fine with most models and datasets. However, moving very large tensors to the GPU can cause out-of-memory errors. That's one of the reasons why we use a dataloader and process the data in batches.
* While the GPU is much faster than the CPU for large matrix operations, **transferring data between the CPU and the GPU is slow**. Therefore, it is best to minimize the number of data transfers between the CPU and the GPU.

## Exercise 3: Prepare the training loop (2 points)

In this exercise, you will prepare the training loop. You should:

1. Initialize the neural network.
2. Define the loss function (CrossEntropyLoss) and the optimizer (Adam).
3. Pass a dictionary with the configuration to wandb. This dictionary should contain all the hyperparameters of our model, including the learning rate, the size of the hidden layers, batch size, etc.
4. Train the neural network. Each epoch should consist of a training and validation phase. You should log the loss and accuracy of the training and validation sets using wandb.
5. Open you project at [wandb.ai](https://wandb.ai/) and see how cool it is!

### Saving and loading the model
As training can take some time, it is a good idea to save the model's state dictionary (its learned weights) to a file after training. You can do this with the following code:

    torch.save(vae.state_dict(), 'vae.pth')
    
To load the model from the file, you can use the following code:

    vae.load_state_dict(torch.load('vae.pth'))

In [1]:
import wandb
wandb.login()

wandb: Currently logged in as: igmure. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/ig/.netrc


True

In [13]:
# Define the configuration dictionary
config = {
    'learning_rate': 0.001,
    'batch_size': 32,
    'epochs': 10,
    'input_size': 784,
    'hidden1_size': 128,
    'hidden2_size': 64,
    'hidden3_size': 32,
    'output_size': 10,
    'dropout_prob': 0.25
}

# Initialize wandb with the configuration
wandb.init(project='mnist-classifier', config=config)

# Initialize the model, loss function, and optimizer
model = NeuralNetwork(
    input_size=config['input_size'],
    hidden1_size=config['hidden1_size'],
    hidden2_size=config['hidden2_size'],
    hidden3_size=config['hidden3_size'],
    output_size=config['output_size'],
    dropout_prob=config['dropout_prob']
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])


# Training loop

In [14]:
# Training loop with wandb logging
def train(model, train_loader, val_loader, epochs=10):
    model.train()
    for epoch in range(epochs):
        train_loss = 0.0
        train_correct = 0
        total_train = 0
        
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(y_pred.data, 1)
            _, labels = torch.max(y_batch.data, 1)
            train_correct += (predicted == labels).sum().item()
            total_train += labels.size(0)
        
        train_loss /= len(train_loader)
        train_accuracy = train_correct / total_train
        
        model.eval()
        val_loss = 0.0
        val_correct = 0
        total_val = 0
        
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                y_pred = model(X_batch)
                loss = criterion(y_pred, y_batch)
                
                val_loss += loss.item()
                _, predicted = torch.max(y_pred.data, 1)
                _, labels = torch.max(y_batch.data, 1)
                val_correct += (predicted == labels).sum().item()
                total_val += labels.size(0)
        
        val_loss /= len(val_loader)
        val_accuracy = val_correct / total_val
        
        wandb.log({
            'epoch': epoch + 1,
            'train_loss': train_loss,
            'train_accuracy': train_accuracy,
            'val_loss': val_loss,
            'val_accuracy': val_accuracy
        })
        
        print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')
        
    wandb.finish()

# Call the train function
train(model, train_dataloader, test_dataloader, epochs=config['epochs'])

Epoch 1/10, Train Loss: 1.6642, Train Accuracy: 0.8063, Val Loss: 1.5371, Val Accuracy: 0.9253
Epoch 2/10, Train Loss: 1.5270, Train Accuracy: 0.9352, Val Loss: 1.5179, Val Accuracy: 0.9439
Epoch 3/10, Train Loss: 1.5105, Train Accuracy: 0.9511, Val Loss: 1.5127, Val Accuracy: 0.9481
Epoch 4/10, Train Loss: 1.5021, Train Accuracy: 0.9595, Val Loss: 1.5076, Val Accuracy: 0.9546
Epoch 5/10, Train Loss: 1.4978, Train Accuracy: 0.9636, Val Loss: 1.5005, Val Accuracy: 0.9599
Epoch 6/10, Train Loss: 1.4932, Train Accuracy: 0.9681, Val Loss: 1.4951, Val Accuracy: 0.9662
Epoch 7/10, Train Loss: 1.4917, Train Accuracy: 0.9697, Val Loss: 1.4984, Val Accuracy: 0.9625
Epoch 8/10, Train Loss: 1.4899, Train Accuracy: 0.9713, Val Loss: 1.4921, Val Accuracy: 0.9691
Epoch 9/10, Train Loss: 1.4883, Train Accuracy: 0.9728, Val Loss: 1.4921, Val Accuracy: 0.9693
Epoch 10/10, Train Loss: 1.4884, Train Accuracy: 0.9728, Val Loss: 1.4908, Val Accuracy: 0.9700


epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▆▇▇██████
train_loss,█▃▂▂▁▁▁▁▁▁
val_accuracy,▁▄▅▆▆▇▇███
val_loss,█▅▄▄▂▂▂▁▁▁
epoch,10
train_accuracy,0.97282
train_loss,1.48845
val_accuracy,0.97
val_loss,1.49078


## Exercise 4: Easy hyperparameter tuning with wandb (2 points)

Wandb allows you to perform hyperparameter tuning by automatically creating multiple runs with different hyperparameters and logging the performance of each run. Below is a brief instruction to `wandb` hyperparameter tuning, but you are more than welcome to find more information in the [official wandb guide](https://docs.wandb.ai/guides/sweeps/).

Your task is to use wandb to perform hyperparameter tuning of the neural network, trying different values of the learning rate, batch size, and the size of the hidden layers. You can use the following hyperparameters:

First, we need to define a dictionary with the hyperparameters that we want to tune. For example:

```python
parameters = {
    'learning_rate': {'values': [0.01, 0.001, 0.0001]},
    'batch_size': {'values': [32, 64, 128]},
    'layer_1_size': {'values': [64, 128, 256]},
    'layer_2_size': {'values': [32, 64, 128]}
}
```

Then we need to create a dictionary with the configuration of the run:

```python
sweep_config = {
    'name': 'mnist-sweep',
    'method': 'grid',   # grid search, you can also try 'random' or 'bayes'
    'metric': {'goal': 'minimize', 'name': 'val_loss'},
    'parameters': parameters,   # that's the dictionary with the hyperparameters
}
```

Finally, we can use the `wandb.sweep` function to perform hyperparameter tuning:

```python
sweep_id = wandb.sweep(sweep_config, project='mnist-classifier')
```

After that, we can finally run the sweep:

```python
wandb.agent(sweep_id, function=train)
```
where `train` is a function that trains the model and logs the metrics to wandb. This function should take a `config` argument, which will contain the hyperparameters of the run. That is how wandb knows which hyperparameters to tune.

1. Rewrite the VAE training loop into a function that takes a single dictionary `parameters` as an argument, initializes the model, optimizer, and criterion, and trains the model for a fixed number of epochs. The function should log the loss and accuracy of the training and validation sets to wandb.
2. Create a dictionary with the hyperparameters that you want to tune.
3. Create a sweep configuration dictionary.
4. Run the sweep and monitor the results in the wandb dashboard.

In [31]:
def train():
    # Initialize the model, loss function, and optimizer
    wandb.init(project='mnist-classifier')
    parameters = wandb.config
    model = NeuralNetwork(
        input_size=parameters.input_size,
        hidden1_size=parameters.hidden1_size,
        hidden2_size=parameters.hidden2_size,
        hidden3_size=parameters.hidden3_size,
        output_size=parameters.output_size,
        dropout_prob=parameters.dropout_prob
    ).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=parameters.learning_rate)


    # Training loop
    model.train()
    for epoch in range(parameters.epochs):
        train_loss = 0.0
        train_correct = 0
        total_train = 0

        for X_batch, y_batch in train_dataloader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(y_pred.data, 1)
            _, labels = torch.max(y_batch.data, 1)
            train_correct += (predicted == labels).sum().item()
            total_train += labels.size(0)

        train_loss /= len(train_dataloader)
        train_accuracy = train_correct / total_train

        model.eval()
        val_loss = 0.0
        val_correct = 0
        total_val = 0

        with torch.no_grad():
            for X_batch, y_batch in test_dataloader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                y_pred = model(X_batch)
                loss = criterion(y_pred, y_batch)

                val_loss += loss.item()
                _, predicted = torch.max(y_pred.data, 1)
                _, labels = torch.max(y_batch.data, 1)
                val_correct += (predicted == labels).sum().item()
                total_val += labels.size(0)

        val_loss /= len(test_dataloader)
        val_accuracy = val_correct / total_val

        wandb.log({
            'epoch': epoch + 1,
            'train_loss': train_loss,
            'train_accuracy': train_accuracy,
            'val_loss': val_loss,
            'val_accuracy': val_accuracy
        })

        print(f'Epoch {epoch + 1}/{parameters["epochs"]}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

    wandb.finish()

In [32]:
parameters = {
    'input_size': {'values': [784]},
    'output_size': {'values': [10]},
    'learning_rate': {'values': [0.01, 0.001, 0.0001]},
    'batch_size': {'values': [16, 32, 64, 128]},
    'hidden1_size': {'values': [64, 128, 256]},
    'hidden2_size': {'values': [32, 64, 128]},
    'hidden3_size': {'values': [16, 32, 64]},
    'dropout_prob': {'values': [0.0, 0.25, 0.5]},
    'epochs': {'values': [5, 10, 15]}
}

sweep_config = {
    'name': 'mnist-sweep',
    'method': 'bayes', # try 'grid' or 'random'
    'metric': {'goal': 'maximize', 'name': 'val_accuracy'}, # if we want to maximize the accuracy
    # remember to log the metric that you want to maximize or minimize!
    'parameters': parameters,
}

sweep_id = wandb.sweep(sweep_config, project='mnist-classifier')    # This will create a new sweep
wandb.agent(sweep_id, function=train)   # This will start the hyperparameter tuning process

Create sweep with ID: dqaqwep8
Sweep URL: https://wandb.ai/igmure/mnist-classifier/sweeps/dqaqwep8


wandb: Agent Starting Run: crqnsdxz with config:
wandb: 	batch_size: 64
wandb: 	dropout_prob: 0.5
wandb: 	epochs: 5
wandb: 	hidden1_size: 64
wandb: 	hidden2_size: 64
wandb: 	hidden3_size: 64
wandb: 	input_size: 784
wandb: 	learning_rate: 0.0001
wandb: 	output_size: 10


Epoch 1/5, Train Loss: 2.1100, Train Accuracy: 0.3693, Val Loss: 1.8088, Val Accuracy: 0.6664
Epoch 2/5, Train Loss: 1.7032, Train Accuracy: 0.7792, Val Loss: 1.6050, Val Accuracy: 0.8748
Epoch 3/5, Train Loss: 1.5878, Train Accuracy: 0.8873, Val Loss: 1.5688, Val Accuracy: 0.9014
Epoch 4/5, Train Loss: 1.5642, Train Accuracy: 0.9062, Val Loss: 1.5532, Val Accuracy: 0.9159
Epoch 5/5, Train Loss: 1.5511, Train Accuracy: 0.9172, Val Loss: 1.5450, Val Accuracy: 0.9226


epoch,▁▃▅▆█
train_accuracy,▁▆███
train_loss,█▃▁▁▁
val_accuracy,▁▇▇██
val_loss,█▃▂▁▁
epoch,5
train_accuracy,0.91725
train_loss,1.55113
val_accuracy,0.92259
val_loss,1.54501


wandb: Agent Starting Run: ehh5qkiy with config:
wandb: 	batch_size: 32
wandb: 	dropout_prob: 0.5
wandb: 	epochs: 10
wandb: 	hidden1_size: 128
wandb: 	hidden2_size: 128
wandb: 	hidden3_size: 32
wandb: 	input_size: 784
wandb: 	learning_rate: 0.0001
wandb: 	output_size: 10


...

wandb: Agent Starting Run: e7ifi15n with config:
wandb: 	batch_size: 128
wandb: 	dropout_prob: 0.5
wandb: 	epochs: 15
wandb: 	hidden1_size: 256
wandb: 	hidden2_size: 64
wandb: 	hidden3_size: 64
wandb: 	input_size: 784
wandb: 	learning_rate: 0.0001
wandb: 	output_size: 10


Epoch 1/15, Train Loss: 1.9956, Train Accuracy: 0.5081, Val Loss: 1.6904, Val Accuracy: 0.7939
Epoch 2/15, Train Loss: 1.6153, Train Accuracy: 0.8612, Val Loss: 1.5692, Val Accuracy: 0.9019
Epoch 3/15, Train Loss: 1.5605, Train Accuracy: 0.9084, Val Loss: 1.5482, Val Accuracy: 0.9199
Epoch 4/15, Train Loss: 1.5423, Train Accuracy: 0.9250, Val Loss: 1.5367, Val Accuracy: 0.9295
Epoch 5/15, Train Loss: 1.5303, Train Accuracy: 0.9353, Val Loss: 1.5292, Val Accuracy: 0.9365
Epoch 6/15, Train Loss: 1.5206, Train Accuracy: 0.9449, Val Loss: 1.5188, Val Accuracy: 0.9456
Epoch 7/15, Train Loss: 1.5137, Train Accuracy: 0.9511, Val Loss: 1.5141, Val Accuracy: 0.9495
Epoch 8/15, Train Loss: 1.5082, Train Accuracy: 0.9566, Val Loss: 1.5093, Val Accuracy: 0.9540
Epoch 9/15, Train Loss: 1.5039, Train Accuracy: 0.9602, Val Loss: 1.5057, Val Accuracy: 0.9578


wandb: Ctrl + C detected. Stopping sweep.


Epoch 10/15, Train Loss: 1.5000, Train Accuracy: 0.9637, Val Loss: 1.5035, Val Accuracy: 0.9594
Epoch 11/15, Train Loss: 1.4967, Train Accuracy: 0.9670, Val Loss: 1.5012, Val Accuracy: 0.9613
Epoch 12/15, Train Loss: 1.4941, Train Accuracy: 0.9698, Val Loss: 1.5006, Val Accuracy: 0.9616
Epoch 13/15, Train Loss: 1.4916, Train Accuracy: 0.9720, Val Loss: 1.4983, Val Accuracy: 0.9639
Epoch 14/15, Train Loss: 1.4892, Train Accuracy: 0.9744, Val Loss: 1.4969, Val Accuracy: 0.9646
Epoch 15/15, Train Loss: 1.4873, Train Accuracy: 0.9761, Val Loss: 1.4958, Val Accuracy: 0.9666


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_accuracy,▁▆▇▇▇██████████
train_loss,█▃▂▂▂▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▅▆▆▇▇▇▇███████
val_loss,█▄▃▂▂▂▂▁▁▁▁▁▁▁▁
epoch,15
train_accuracy,0.97607
train_loss,1.48728
val_accuracy,0.9666
val_loss,1.49577
